## Part 1

In [1]:
train = open("data/train", "r")
lines = train.readlines()

In [2]:
for line in lines[:10]:
    print(line.split('\t'))

['1', 'Pierre', 'NNP\n']
['2', 'Vinken', 'NNP\n']
['3', ',', ',\n']
['4', '61', 'CD\n']
['5', 'years', 'NNS\n']
['6', 'old', 'JJ\n']
['7', ',', ',\n']
['8', 'will', 'MD\n']
['9', 'join', 'VB\n']
['10', 'the', 'DT\n']


In [3]:
vocab = {}
for i in range(len(lines)):
    if lines[i] == '\n':
        continue
    
    line = lines[i].split('\t')
    if line[1] not in vocab:
        vocab[line[1]] = 1
    else:
        vocab[line[1]] = 1 + vocab[line[1]]    

In [4]:
threshold = 2
unk_dict = {'< unk >': 0}
   
for word in vocab:
    if vocab[word] <= threshold:
        unk_dict['< unk >'] = vocab[word] + unk_dict['< unk >']

In [5]:
final_vocab = vocab.copy()
for word in vocab:
    if final_vocab[word] <= threshold:
        del final_vocab[word]
#     else:
#         del unk_total_dict[word]

In [6]:
final_vocab = sorted(final_vocab.items(), key=lambda x:x[1], reverse = True)
final_vocab = dict(final_vocab)

In [7]:
final_vocab = list(final_vocab.items())
final_vocab.insert(0, ('< unk >', unk_dict['< unk >']))

In [8]:
file = open("vocab.txt", "w")

In [9]:
for i in range(len(final_vocab)):
    file.write(final_vocab[i][0])
    file.write('\t')
    file.write(str(i))
    file.write('\t')
    file.write(str(final_vocab[i][1]))
    file.write('\n')

In [10]:
print("Total size of vocabulary is:", len(final_vocab))

Total size of vocabulary is: 16920


In [11]:
print("The total occurrences of the special token ‘< unk >’ after replacement is:", final_vocab[0][1])

The total occurrences of the special token ‘< unk >’ after replacement is: 32537


## Part 2

In [12]:
file1 = open("hmm.json", "w")

In [13]:
tag_freq = {}
for i in range(len(lines)):
    if i == 0:
        tag_freq['start_tag'] = 1 + tag_freq.get('start_tag', 0)
    elif lines[i] == '\n':
        tag_freq['start_tag'] = 1 + tag_freq.get('start_tag', 0)
        continue
    line = lines[i].split('\t')
    if line[2][:-1] not in tag_freq:
        tag_freq[line[2][:-1]] = 1
    else:
        tag_freq[line[2][:-1]] = 1 + tag_freq[line[2][:-1]]

In [14]:
tag_tag_freq = {}
for i in range(len(lines)-1):
    if lines[i] != '\n' and lines[i+1] != '\n':
        tag1 = lines[i].split('\t')[2][:-1]
        tag2 = lines[i+1].split('\t')[2][:-1]
        key = (tag1, tag2)

        tag_tag_freq[key] = 1 + tag_tag_freq.get(key, 0)

In [15]:
tag_list = []
tag_list.append('start_tag')
for i in range(len(lines)):
    if lines[i] != '\n':
        tag_list.append(lines[i].split('\t')[2][:-1])
    else:
        tag_list.append('start_tag')

In [16]:
transition = {}
for tag1 in tag_freq:
    for tag2 in tag_freq:
        count = 0
        for k in range(len(tag_list)-1):
            if tag_list[k] == tag1 and tag_list[k+1] == tag2:
                count += 1
        
        transition[(tag1, tag2)] = count/tag_freq[tag1]

transition_str = {}
temp = list(transition.items())
for i in range(len(transition)):
    k, v = temp[i]
    k = f"{k}"
    transition_str[k] = v

In [17]:
total_dict = {}
for i in range(len(lines)):
    if lines[i] == '\n':
        continue
    line = lines[i].split('\t')
    word = line[1]
    tag = line[2][:-1]
    if vocab[word] <= threshold:
        word = "< unk >"
    key = (tag, word)
    total_dict[key] = 1 + total_dict.get(key, 0)

In [18]:
final_vocab = dict(final_vocab)

In [19]:
tag_word_list = []
tag_word_list.append('start_tag', )
for i in range(len(lines)):
    if lines[i] != '\n':
        tag_list.append(lines[i].split('\t')[2][:-1])
    else:
        tag_list.append('start_tag')

In [20]:
emission1 = {}
for tag in tag_freq:
    if tag != 'start_tag':
        for word in final_vocab:
            key1 = (tag, word)
            if key1 in total_dict:
                emission1[key1] = total_dict[key1]/tag_freq[tag]
            else:
                emission1[key1] = 0
        
        
emission1_str = {}
temp = list(emission1.items())
for i in range(len(emission1)):
    k, v = temp[i]
    k = f"{k}"
    emission1_str[k] = v

In [21]:
model = {'transition':transition_str,'emission':emission1_str}

In [22]:
import json

with open("hmm.json", "w") as outfile:
    json.dump(model, outfile)

In [23]:
print("Transition parameters:", len(transition))
print("Emission parameters:", len(emission1))

Transition parameters: 2116
Emission parameters: 761400


## Part 3

In [24]:
def greedy_algo(lines, tag_data, t1, e1): 
    prediction = []
    for i in range(len(lines)):
        pred = []
        for j in range(len(lines[i])):     
            word = lines[i][j]
            tag_ans = ''
            ans = float('-inf')
 
            for tag1 in tag_freq:
                if tag1 != 'start_tag':
                    x = 'start_tag' if j == 0 else pred[-1]
                    key1 = (x, tag1)
                    key2 = (tag1, word)
                    if word not in final_vocab:
                        key2 = (tag1, '< unk >')
                    
                    prob = t1[key1] * e1[key2]
                    ans = max(prob, ans)
                    if ans == prob:
                        tag_ans = tag1
            pred.append(tag_ans)
        prediction.append(pred)
    return prediction

In [25]:
dev = open("data/dev", "r")
dev_lines = dev.readlines()
sentences = []
dev_true = []
sentence = []
dev_true_single = []
for line in dev_lines:
    if line != '\n':
        sentence.append(line.split('\t')[1])
        dev_true_single.append(line.split('\t')[2][:-1])
    else:
        sentences.append(sentence)
        dev_true.append(dev_true_single)
        sentence = []
        dev_true_single = []

In [26]:
dev_pred = greedy_algo(sentences, tag_freq, transition, emission1)

In [27]:
count = 0
wordss = 0
for i in range(len(dev_true)):
    for j in range(len(dev_true[i])):
        if dev_true[i][j] == dev_pred[i][j]:
            count += 1
    wordss += len(dev_true[i])

    
accuracy = (count/wordss) *100
print("Dev accuracy:", accuracy)

Dev accuracy: 92.88202746089213


In [28]:
test = open("data/test", "r")
test_lines = test.readlines()
sentences = []
sentence = []
for line in test_lines:
    if line != '\n':
        sentence.append(line.split('\t')[1][:-1])
    else:
        sentences.append(sentence)
        sentence = []

In [29]:
test_pred = greedy_algo(sentences, tag_freq, transition, emission1)

In [30]:
file2 = open("greedy.out", "w")

test = open("data/test", "r")
test_lines = test.readlines()
sentences = []
sentence = []
for line in test_lines:
    if line != '\n':
        sentence.append(line.split('\t')[1][:-1])
    else:
        sentences.append(sentence)
        sentence = []
        
for i in range(len(sentences)):
    for j in range(len(sentences[i])):
        idx = str(j+1)
        file2.write(idx)
        file2.write('\t')
        file2.write(sentences[i][j])
        file2.write('\t')
        file2.write(test_pred[i][j])
        file2.write('\n')
    file2.write('\n')

## Part 4

In [31]:
def viterbi_algo(sents, tag_data, transition, emission1):
    preds = []
    for i in range(len(sents)):
        lines = sents[i]
        states = set(list(tag_data))
        states.remove("start_tag")
        trails = {}
        for s in states:
            key1 = ("start_tag", s)
            key2 = (s, lines[0])
            if lines[0] not in final_vocab:
                key2 = (s, '< unk >')
            trails[s, 0] = transition[key1] * emission1[key2]
        for o in range(1, len(lines)):
            obs = lines[o]
            if obs not in final_vocab:
                obs = '< unk >'
            for s in states:
                v1 = [(trails[k, o - 1] * transition[(k, s)] * emission1[(s, obs)], k) for k in states]
                k = sorted(v1)[-1][1]
                trails[s, o] = trails[k, o - 1] * transition[(k, s)] * emission1[(s, obs)]
        best_path = []
        for o in range(len(lines) - 1, -1, -1):
            k = sorted([(trails[k, o], k) for k in states])[-1][1]
            best_path.append(k)
        best_path.reverse()
        preds.append(best_path)

    return preds

In [32]:
dev = open("data/dev", "r")
dev_lines = dev.readlines()
sentences = []
dev_true = []
sentence = []
dev_true_single = []
for line in dev_lines:
    if line != '\n':
        sentence.append(line.split('\t')[1])
        dev_true_single.append(line.split('\t')[2][:-1])
    else:
        sentences.append(sentence)
        dev_true.append(dev_true_single)
        sentence = []
        dev_true_single = []

In [33]:
dev_pred = viterbi_algo(sentences, tag_freq, transition, emission1)

In [34]:
count = 0
wordss = 0
for i in range(len(dev_true)):
    for j in range(len(dev_true[i])):
        if dev_true[i][j] == dev_pred[i][j]:
            count += 1
    wordss += len(dev_true[i])

    
accuracy = (count/wordss) * 100
print("Dev accuracy:", accuracy)

Dev accuracy: 93.11883780768268


In [35]:
test = open("data/test", "r")
test_lines = test.readlines()
sentences = []
sentence = []
for line in test_lines:
    if line != '\n':
        sentence.append(line.split('\t')[1][:-1])
    else:
        sentences.append(sentence)
        sentence = []

In [36]:
test_pred = viterbi_algo(sentences, tag_freq, transition, emission1)

In [37]:
file3 = open("viterbi.out", "w")
test = open("data/test", "r")
test_lines = test.readlines()

sentences = []
sentence = []
for line in test_lines:
    if line != '\n':
        sentence.append(line.split('\t')[1][:-1])
    else:
        sentences.append(sentence)
        sentence = []
        
for i in range(len(sentences)):
    for j in range(len(sentences[i])):
        idx = str(j+1)
        file3.write(idx)
        file3.write('\t')
        file3.write(sentences[i][j])
        file3.write('\t')
        file3.write(test_pred[i][j])
        file3.write('\n')
    file3.write('\n')

In [38]:
file.close()
file1.close()
file2.close()
file3.close()